In [2]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv('mainIndex.csv')
# df1 = pd.read_csv('2000-2020.csv')
# df2 = pd.read_csv('2021.csv')
# df3 = pd.read_csv('2022.csv')
# df4 = pd.read_csv('2023.csv')
# df5 = pd.read_csv('2024.csv')
# df_cleaned = pd.read_csv('cleaned_data.csv')
# df_all = pd.concat([df1,df2,df3,df4,df5], axis=0)

In [24]:
df

,Publication Number,Application Number,Application Date,Publication Date,IPC - Current,Claims (English),Claims Count,Assignee - Original,Inventor First,Inventor - Original,...,Inventor Count,Count of Cited Refs - Non-patent,Cited Refs - Non-patent,Count of Citing Patents,DWPI Family Members,DWPI Count of Family Members,DWPI Count of Family Countries/Regions,IPC Count,Assignee Standardized,Inventor Standardized
0,WO2000072172A1,WO2000US14095A,2000-05-23,2000-11-30,G06Q003006,CLAIMS \n | 1. A collective business system co...,28.0,CLARK Douglas W.,"CLARK, Douglas, W.","CLARK, Douglas, W.",...,1,0,NaN,1,WO2000072172A1 | AU200051544A | US6351738B1 | ...,4.0,91.0,1,CLARK DOUGLAS W,CLARK DOUGLAS W
1,CA2377149A1,CA2377149A,2000-05-23,2000-11-30,G06Q003006,CLAIMS What is claimed is: \n | 1. A collectiv...,28.0,CLARK DOUGLAS W.,"CLARK, DOUGLAS W.","CLARK, DOUGLAS W.",...,1,0,NaN,0,NaN,NaN,NaN,1,CLARK DOUGLAS W,CLARK DOUGLAS W
2,US6192347B1,US1998134451A,1998-08-14,2001-02-20,G06Q004000,What is claimed is: \n | 1. A method for maki...,128.0,Graff/Ross Holdings,"Graff, Richard A.","Graff, Richard A.",...,1,16,"Auster, ""Amortizing Life Estates and Term Inte...",386,US6192347B1 | US6192347C1,2.0,1.0,1,GRAFFROSS HOLDINGS,GRAFF RICHARD A
3,WO2001077925A1,WO2000KR1441A,2000-12-13,2001-10-18,G06F001718 | G06Q004000 | G06Q004006,Claims \n | 1. An investment consulting system...,12.0,KIM Kee-woung,"CHANG, Uk","CHANG, Uk | KIM, In-young | PARK, Jong-won",...,3,0,NaN,1,WO2001077925A1 | AU200120280A | KR2001093900A ...,12.0,92.0,2,KIM KEE WOUNG,CHANG UK | KIM IN YOUNG | PARK JONG WON
4,JP2002007703A,JP2000182738A,2000-06-19,2002-01-11,G06Q004006 | G06Q004000 | G06Q004002 | G06Q004004,|1. The computerized financial transaction sys...,8.0,SUMISHO LEASE CO LTD,OKAMOTO TORU,OKAMOTO TORU,...,1,0,NaN,16,JP2002007703A,1.0,1.0,1,SUMISHO LEASE CO LTD,OKAMOTO TORU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,KR2820754B1,KR20227011127A,2020-09-08,2025-06-13,H04W003600 | H04L00651016 | H04W003614 | H04W0...,|1. NG-RAN (Next Generation Radio Access Netwo...,19.0,LG Electronics Inc.,Myungjune YOUN,"Myungjune YOUN | LEE, Se Hui | CHUNSung Duck |...",...,6,2,3GPP TS23.501 v16.1.0,0,WO2021049841A1 | KR2022062557A | CN114600504A ...,10.0,136.0,2,LG ELECTRONICS INC,MYUNGJUNE YOUN | LEE SE HUI | CHUNSUNG DUCK | ...
29995,KR2025086641A,KR20257012452A,2023-10-04,2025-06-13,H04W00720446 | H04L000500 | H04L000514 | H04W0...,|1. Receiving a TDD (Time Division Duplexing) ...,15.0,LG Electronics Inc.,Song Minwoo,"Song Minwoo | KO, Hyunsoo | YOU, Hyangsun | SH...",...,4,0,NaN,0,WO2024072180A1 | CN119968908A | KR2025086641A ...,4.0,140.0,2,LG ELECTRONICS INC,SONG MINWOO | KO HYUNSOO | YOU HYANGSUN | SHIM...
29996,KR2025086624A,KR20257010975A,2023-10-06,2025-06-13,H04W00740808 | H04L002726 | H04W007202 | H04W0...,|1. A method in which the first device perform...,20.0,LG Electronics Inc.,Daesung HWANG,"Daesung HWANG | LEESEUNGMIN | SEO, Hanbyul",...,3,0,NaN,0,WO2024076187A1 | KR2025086624A | CN120153748A ...,4.0,140.0,2,LG ELECTRONICS INC,DAESUNG HWANG | LEESEUNGMIN | SEO HANBYUL
29997,KR2025086588A,KR20257004190A,2023-10-11,2025-06-13,H04W005202 | H04W002408 | H04W006802 | H04W007...,|1. In a method of receiving a signal in a wir...,15.0,LG Electronics Inc.,"HWANG, Seunggye","HWANG, Seunggye | KIM, Seonwook | LEE, Youngda...",...,5,0,NaN,0,WO2024080734A1 | KR2025086588A,2.0,139.0,1,LG ELECTRONICS INC,HWANG SEUNGGYE | KIM SEONWOOK | LEE YOUNGDAE |...


In [26]:
import pandas as pd
import numpy as np

# keep a copy to inspect failures later
orig = df['Application Date'].copy()

# normalize blanks
col = df['Application Date'].astype(str).str.strip().replace({'': pd.NA, 'nan': pd.NA})

# 1) parse mixed string formats
dt = pd.to_datetime(col, format='mixed', dayfirst=True, errors='coerce')

# 2) handle Excel serial numbers if present
mask_num = col.str.fullmatch(r'\d+')
serial = pd.to_numeric(col.where(mask_num), errors='coerce')
excel_dt = pd.to_datetime('1899-12-30') + pd.to_timedelta(serial, unit='D')
dt = dt.fillna(excel_dt)

# 3) drop anything still out of bounds or absurd years
dt = dt.where((dt >= pd.Timestamp('1677-09-21')) & (dt <= pd.Timestamp('2262-04-11')))
dt = dt.where(dt >= pd.Timestamp('1900-01-01'))  # tighten if you want

# assign back
df['Application Date'] = dt

# inspect rows that failed to parse
bad_rows = df[df['Application Date'].isna()][['Application Date']].assign(_raw=orig)
# print(bad_rows.head())  # optional

# your filter
df = df[(df['Application Date'] >= '2000-01-01') & (df['Application Date'] <= '2024-12-31')]

In [27]:
df.shape

(29538, 32)

In [28]:
cols = ['Publication Number','Application Date','Publication Date','IPC Subclass','Assignee Standardized','Claims Count','Assignee Count','Inventor Standardized','Publication Country Code','Count of Cited Refs - Patent','Inventor Count', 'IPC Count','Count of Cited Refs - Non-patent','Count of Citing Patents','DWPI Count of Family Members','DWPI Count of Family Countries/Regions']
df=df[cols]

In [30]:
#Now delete all the columns for which all rows are having value null
df = df.dropna(axis=1, how='all')
print('data shape=',df.shape)

data shape= (29538, 16)


In [31]:
df.isnull().sum()

Publication Number                          0
Application Date                            0
Publication Date                            0
IPC Subclass                              265
Assignee Standardized                     154
Claims Count                               65
Assignee Count                              0
Inventor Standardized                     379
Publication Country Code                    0
Count of Cited Refs - Patent                0
Inventor Count                              0
IPC Count                                   0
Count of Cited Refs - Non-patent            0
Count of Citing Patents                     0
DWPI Count of Family Members              139
DWPI Count of Family Countries/Regions    139
dtype: int64

In [32]:
INDICATORS = [
    "Count of Cited Refs - Patent",
    "Count of Cited Refs - Non-patent",
    "Count of Citing Patents",
    "DWPI Count of Family Members",
    "DWPI Count of Family Countries/Regions",
    "Assignee Count",
    "Inventor Count",
    "Claims Count",
    "IPC Subclass",
    "Assignee Standardized",
    "Inventor Standardized",
    "IPC Count",
]

In [33]:
df = df[df[INDICATORS].notna().all(axis=1)]

In [34]:
df.shape

(28702, 16)

In [35]:
df.to_csv("MainData.csv", index=False)

In [40]:
df_pi = pd.read_csv("../../derwent/merged.csv")

In [42]:
df_pi.shape

(15563, 113)